In [1]:
import setVCD

# setVCD Motivating Example
## Scenario:
Here is an excerpt of the VCD file we use for testing (under `tests/fixtures/wave.vcd`):

![alt text](img/gtkwave.png "Title")

You are debugging an input stream (`io_input`), and would like to see the data only when the transaction is valid. This notebook goes through the steps of doing this with `setVCD`.

## Creating the object
You must specifify the path and name of the clock signal in the file.

In [2]:
# Load VCD file
vcd_path = "./tests/fixtures/wave.vcd"
sv = setVCD.SetVCD(vcd_path, clock="TOP.clk")

## Get signals of interest
First, we get the simulation timesteps when the clock is at a rising edge and when reset is 0. We can do this easily, as the `SetVCD.get` returns a set, with which we can easily use the intersection (`&`) operator with. 

In [3]:
rising_edges = sv.get("TOP.clk", lambda tm1, t, tp1: tm1 == 0 and t == 1)
reset_is_0 = sv.get("TOP.reset", lambda tm1, t, tp1: t == 0)
clock_updates = rising_edges & reset_is_0

# Need to convert set to sorted list before printing it out.
clock_updates_list = sorted(clock_updates)
print(clock_updates_list[0:10])

[34, 36, 38, 40, 42, 44, 46, 48, 50, 52]


We can search the VCD signals for the inputs to our accelerator. 

In [4]:
sv.search("Accelerator.io_input")

['TOP.Accelerator.io_input_valid',
 'TOP.Accelerator.io_input_ready',
 'TOP.Accelerator.io_input_payload_last',
 'TOP.Accelerator.io_input_payload_fragment_value_0[15:0]']

To get the timesteps where the output is valid, we care that the `valid` and `ready` signals are both high, so let's filter them with `get` again. 

In [5]:
# Get times when output_valid and output_ready are asserted.
in_valid = sv.get("TOP.Accelerator.io_input_valid", lambda tm1, t, tp1: t == 1)
in_ready = sv.get("TOP.Accelerator.io_input_ready", lambda tm1, t, tp1: t == 1)

# Get timesteps of valid outputs
valid_input_timesteps = rising_edges & reset_is_0 & in_ready & in_valid

Once We've acquired the valid timesteps, we can get the values from the data signal (`payload_fragment_value`). 

The inputs to the Accelerator module are **fixed point SQ1.15**, so we specify this in the `value_type`.

In [6]:
inputs = sv.get_values(
    "TOP.Accelerator.io_input_payload_fragment_value_0[15:0]",
    valid_input_timesteps,
    value_type=setVCD.FP(frac = 15, signed = True)
)
print(inputs[0:10])

[0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375, 0.5999755859375]


We've filtered down the `input_fragment_value` signal only to timesteps that we care about! We can now create various assertions and inspections on the signal which can significantly improve debugging over simply looking at VCD waves.

In [7]:
# Get (timestep, value) tuples instead of just output
input_with_t = sv.get_values_with_t(
    "TOP.Accelerator.io_input_payload_fragment_value_0[15:0]",
    valid_input_timesteps,
    value_type=setVCD.FP(frac = 15, signed = True)
)

print(f"Received expected (100) inputs? {len(inputs) == 100}")

avg_latency = 0
avg_value = 0
for i in range(1, len(input_with_t)):
    t, v = input_with_t[i]
    tm1, vm1 = input_with_t[i-1]
    avg_latency = (avg_latency + (t - tm1)) / 2
    avg_value = (avg_value + v) / 2

print(f"Average clocks between inputs: {avg_latency:.2f}")
print(f"Average input value: {avg_value:.5f}")

Received expected (100) inputs? True
Average clocks between inputs: 10.38
Average input value: 0.59998
